## Install and Import Modules

In [ ]:
!pip install refinitiv-data

In [39]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd

In [40]:
#open session to connect RD Libraries
rd.open_session()

<refinitiv.data.session.Definition object at 0x132a6a5e0 {name='workspace'}>

## Build Futurs RIC constructor object

#### Inrtroduce AssetExchanges object to store Exchange-Futures pairs and month codes

In [41]:
from dataclasses import dataclass

@dataclass
class AssetExchanges:
    
    data = {
         'EUX': ['FGBS','FOAT','FGBM','FGBL','FGBX','FDX','FBTP','STXE','FSMI','FBTS','FMTW'],
         'CBT': ['BO', 'FV', 'SM','TU','TY','US','W', 'AUL','RR','YC','KW','YM','S'],
         'IEU': ['LCO', 'FLG', 'FFI', 'SON3', 'LGO', 'FEIT', '0FSS'],
         'IMM': ['AD', 'ED', 'NE', 'SF', 'JY', 'BP', 'CD'],
         'IUS': ['DX', 'KC', 'MEM', 'CC', 'CT', 'SB'],
         'NYM': ['S', 'PA', 'PL', 'RB', 'HO', 'CL'],
         'IOM': ['NIY', 'NQ', 'RTY', 'ES'],
         'BMF': ['DOL', 'DIJ', 'IND'],
         'SFE': ['YBA', 'YTC', 'YAP'],
         'SIM': ['STWN', 'SIN', 'SFC'],
         'CME': ['FC', 'LH', 'LC'],
         'CMX': ['SI', 'HG', 'GC'],
         'OSA': ['JGB', 'JNI', 'JTI'],
         'KFE': ['10TB', 'KTB', 'KS'],
         'MON': ['BAX', 'CGB', 'SXM'],
         'HFE': ['HCEI', 'HSI'],
         'CBF': ['VX'],
         'EDX': ['CFI2'],
         'GEM': ['MP'],
         'IST': ['XU030'],
         'JNB': ['ALSI'],
         'LME': ['MCU'],
         'MNP': ['FCE'],
         'MRV': ['MFXI'],
         'RTF': ['RIRTS'],    
         'WSE': ['FW2020']}
    month_codes =  {'F':'01', 'G':'02', 'H':'03', 'J':'04', 'K':'05', 'M':'06', 'N':'07', 'Q':'08', 'U':'09', 'V':'10', 'X':'11', 'Z':'12'}

#### Introduce FururesRICs object for RIC construction

In [42]:
class FuturesRICs:
    def __init__(self, asset_exchanges):
        self.asset_exchanges =  AssetExchanges.data
        self.month_codes = AssetExchanges.month_codes
        
    def get_future_ric(self, underlying, month_code, year):
        exchange = self._get_exchange_code(underlying)
        if exchange:
            
            query = f'{underlying}{month_code}{str(year)[-1]}*'
            filter_criteria = f"ExchangeCode eq '{exchange}' and RCSAssetCategoryLeaf eq 'Future'" \
            +  f" and ExpiryDate ge {year-1}-{self.month_codes[month_code]}-28 and ExpiryDate le {year}-{self.month_codes[month_code]}-28 " \
            +  f" and RIC xeq '{query[:-1]}' or RIC xeq '{query[:-1]}^{str(year)[-2]}'"
            
            response = self._search_future_ric(query, filter_criteria)
            if not response.empty:
                return response
            else:
                print(f'No future with specified parameters for {underlying}')
        else:
            print(f'No underlying exchange is set for {underlying}')
    
    def _search_future_ric(self, query, filter_criteria):

        response = search.Definition(
        view = search.Views.SEARCH_ALL,
        query=query,
        select="DocumentTitle, RIC, ExchangeCode, ExpiryDate, RCSUnderlyingMarket" +
                "UnderlyingQuoteName, UnderlyingQuoteRIC, RCSAssetCategoryLeaf, RetireDate",
        filter=filter_criteria).get_data().data.df
        
        return response
    
    def _get_exchange_code(self, underlying):
        for exchange, asset in self.asset_exchanges.items():
            if underlying in asset:
                return exchange

## Initilize the FuturesRICs object and get futures RICs

#### Construct futures RIC

In [43]:
fr = FuturesRICs(AssetExchanges)

In [ ]:
fr.get_future_ric('LCO', 'H', 2022)

#### Get pricing data

In [38]:
rd.get_history(
    universe=['LCOH2^2'],
    # fields=["TRDPRC_1"],
    interval="1D",
    start="2022-02-01",
    end="2022-03-30",
)

""


## Experiment with multiple futures for different years and months

In [7]:
# load futures list from excel
futures = pd.read_excel('futures.xlsx')

In [33]:
# get the future chains from instrument sheet to derive underlying RICs
futures_chains= futures.loc[futures['Instrument'] == 'FUTURE' ]['RIC'].to_list()

In [34]:
len(futures_chains)

91

In [35]:
# get underlyings from future chains
import re
s = '0#AD:'
underlyings = []
for ric in futures_chains:
    underlyings.append(re.search(r'#(.*?):', ric).group(1))

In [36]:
len(underlyings)

91

In [ ]:
#test the results - this is mostly done to make sure we are not getting several responses for any of the underlying for the specified periods
months = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z']

several_responses = []
no_response = []
rics = []
for underlying in underlyings:
    
    for i in range(2010, 2023):
        print(i)
        for month in months:
            try:
                response = fr.get_future_ric(underlying, 'H', 2022)
                if response is not None:
                    if len(response) == 1:
                        rics.append(response['RIC'][0])
                    elif len(response) > 1:
                        several_responses.append(underlying)
                else:
                    print(underlying, 'No response')
                    no_response.append(underlying)
            except:
                continue
    print(underlying, len(rics), len(several_responses))

In [37]:
import numpy as np
np.unique(np.array(no_response))

array(['DOL', 'FSS', 'IND', 'MCU', 'NG', 'STW', 'XU030'], dtype='<U5')